# 匯入函式庫

In [10]:
from collections import Counter
import jieba
jieba.set_dictionary('dict.txt')  # 如果是使用繁體文字，請記得去下載繁體字典來使用
import numpy as np
import pandas as pd
from numpy.linalg import norm

# 匯入資料

In [11]:
# 把檔案讀出來(原始資料: https://society.hccg.gov.tw/ch/home.jsp?id=43&parentpath=0,5)
df_QA = pd.read_json('ProcessedData.json', encoding='utf8')
# 使用到question跟ans這兩個欄位
df_question = df_QA[['question', 'ans']].copy()  ## 不要更動到原始的DataFrame
df_question.drop_duplicates(inplace=True)  ## 丟掉重複的資料
df_question.head(5)  ## show出來

,question,ans
0,小孩出生後應於何時申請育兒津貼?,1.幼兒家長在戶政事務所完成新生兒出生登記後，即可向所轄區公所社政課提出育兒津貼申請。2.在...
1,小孩出生後應於何時申請育兒津貼?,隨時提出;津貼經審查通過後，追溯自受理申請之當月起發給。兒童出生後六十日內向戶政事務所完成出...
2,育兒津貼申請應備文件為何?,申請資料應備齊:(一)兒童之戶口名簿影本。(二)申請人之郵局存摺封面影本。(三)父母雙方身分...
3,若民眾夫妻雙方均失業，是否可申請家庭育兒津貼費用補助,一、育兒津貼補助對象：1.育有二足歲以下兒童。2.兒童之父母至少一方因育兒需要，致未能就業者...
4,育兒津貼補助對象為何？,育兒津貼補助對象，應符合下列規定：(一)育有二足歲以下兒童。(二)兒童之父母(或監護人)至少...


# 資料前處理

In [12]:
#前處理
all_terms = []
def preprocess(item):  ##定義前處理的function
    terms = [t for t in jieba.cut(item, cut_all=True)]  ## 把全切分模式打開，可以比對的詞彙比較多
    all_terms.extend(terms)  ## 收集所有出現過的字
    return terms

df_question['processed'] = df_question['question'].apply(preprocess)
print(df_question.iloc[0])

df_question.head()

Building prefix dict from C:\Users\ntut17\Desktop\NLP\dict.txt ...
Loading model from cache C:\Users\ntut17\AppData\Local\Temp\jieba.u117617a9354c14946ff2f60642613b5f.cache
Loading model cost 0.434 seconds.
Prefix dict has been built successfully.


question                                      小孩出生後應於何時申請育兒津貼?
ans          1.幼兒家長在戶政事務所完成新生兒出生登記後，即可向所轄區公所社政課提出育兒津貼申請。2.在...
processed                 [小孩, 出生, 後, 應, 於, 何時, 申請, 育兒, 津貼, ?]
Name: 0, dtype: object


,question,ans,processed
0,小孩出生後應於何時申請育兒津貼?,1.幼兒家長在戶政事務所完成新生兒出生登記後，即可向所轄區公所社政課提出育兒津貼申請。2.在...,"[小孩, 出生, 後, 應, 於, 何時, 申請, 育兒, 津貼, ?]"
1,小孩出生後應於何時申請育兒津貼?,隨時提出;津貼經審查通過後，追溯自受理申請之當月起發給。兒童出生後六十日內向戶政事務所完成出...,"[小孩, 出生, 後, 應, 於, 何時, 申請, 育兒, 津貼, ?]"
2,育兒津貼申請應備文件為何?,申請資料應備齊:(一)兒童之戶口名簿影本。(二)申請人之郵局存摺封面影本。(三)父母雙方身分...,"[育兒, 津貼, 申請, 應, 備, 文件, 為何, ?]"
3,若民眾夫妻雙方均失業，是否可申請家庭育兒津貼費用補助,一、育兒津貼補助對象：1.育有二足歲以下兒童。2.兒童之父母至少一方因育兒需要，致未能就業者...,"[若, 民眾, 夫妻, 雙方, 均, 失業, ，, 是否, 可, 申請, 家庭, 育兒, 津..."
4,育兒津貼補助對象為何？,育兒津貼補助對象，應符合下列規定：(一)育有二足歲以下兒童。(二)兒童之父母(或監護人)至少...,"[育兒, 津貼, 貼補, 補助, 對象, 為何, ？]"


# 建立詞彙表

In [13]:
# 建立termindex: 將all_terms取出不重複的詞彙，並轉換型別為list(避免順序亂掉)
termindex = list(set(all_terms))

print("len(termindex)", len(termindex))
print(termindex[:10])

len(termindex) 1038
['', '餐食', '放', '傷病', '似乎', '加害', '暫行', '租金', '留職', '弱勢']


# 計算IDF

In [14]:
# 建立IDF vector
Doc_Length = len(df_question)  ## 計算出共有幾篇文章
Idf_vector = []  ## 初始化IDF向量
for term in termindex:  ## 對index中的詞彙跑回圈
    num_of_doc_contains_term = 0  ## 計算有機篇文章出現過這個詞彙
    for terms in df_question['processed']:
        if term in terms:
            num_of_doc_contains_term += 1
    idf = np.log(Doc_Length/num_of_doc_contains_term)  ## 計算該詞彙的IDF值
    Idf_vector.append(idf)
print(len(Idf_vector))
print(termindex[:10])
print(Idf_vector[:10])

1038
['', '餐食', '放', '傷病', '似乎', '加害', '暫行', '租金', '留職', '弱勢']
[4.177459468932607, 5.093750200806762, 5.786897381366708, 5.786897381366708, 5.786897381366708, 3.8409872323113943, 5.786897381366708, 5.786897381366708, 5.786897381366708, 4.688285092698598]


# 計算TF

In [15]:
# 建立document vector
def terms_to_vector(terms):  ## 定義把terms轉換成向量的function
    vector = np.zeros_like(termindex, dtype=np.float32)  
    
    for term, count in Counter(terms).items():
        # 計算vector上每一個字的tf值
        vector[termindex.index(term)] = count
    # 計算tfidf，element-wise的將vector與Idf_vector相乘
    vector = vector * Idf_vector
    return vector



df_question['vector'] = df_question['processed'].apply(terms_to_vector)  ## 將上面定義的function，套用在每一筆資料的terms欄位上
df_question['vector'][100:120]

100    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
101    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
102    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
103    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
104    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
105    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
106    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
107    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
108    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
109    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
110    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
111    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
112    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
113    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
114    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
115    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
116    [0.0, 5.093750200806762, 0.0, 0.0, 0.0, 0.0, 0...
117    [0.0, 0.0, 0.0, 0.0, 0.0

# 計算餘弦相似性

In [16]:


def cosine_similarity(vector1, vector2):  ## 定義cosine相似度的計算公式
    score = np.dot(vector1, vector2)  / (norm(vector1) * norm(vector2))
    return score#越大越像

sentence1 = df_question.loc[0]  ##取出第零個的問題
sentence2 = df_question.loc[2]  ##取出第二個的問題
sentence10 = df_question.loc[10]
print(sentence1['question'])
print(sentence2['question'])
print(sentence10['question'])

print(cosine_similarity(sentence1['vector'], sentence2['vector']))  ##計算兩者的相似度
print(cosine_similarity(sentence1['vector'], sentence10['vector']))  ##計算兩者的相似度


小孩出生後應於何時申請育兒津貼?
育兒津貼申請應備文件為何?
親戚朋友托育，是否也可以領補助呢？
0.3123460493778919
0.0


# 資料檢索

In [17]:
def retrieve(testing_sentence, return_num=3):  ## 定義出檢索引擎，找出前3相關
    testing_vector = terms_to_vector(preprocess(testing_sentence))  ## 把剛剛的前處理、轉換成向量的function，應用在使用者輸入的問題上
    idx_score_mapping = [(idx, cosine_similarity(testing_vector, vec)) for idx, vec in enumerate(df_question['vector'])]
    top3_idxs = np.array(sorted(idx_score_mapping, key=lambda x:x[1], reverse=True))[:3, 0]
    return df_question.loc[top3_idxs, ['question', 'ans']]

idxs = retrieve("老人年金").index
print(idxs)
# Float64Index([100.0, 111.0, 321.0], dtype='float64')
print(df_question.loc[idxs, 'question'])

Float64Index([100.0, 111.0, 321.0], dtype='float64')
100.0    我已經年滿65歲領有國民年金老人年金及基本保證年金3628元，因家境清寒還可以再申請中低收入...
111.0                            新竹市老人一般可領老人津貼6628元，該如何申請？
321.0           國民年金保險被保險人如果是家庭收入較低者，國民年金保險費是否可以減免？補助標準為何？
Name: question, dtype: object
